In [1]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras

In [2]:
import os

os.environ["KERAS_BACKEND"] = "torch"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [3]:
import keras
import keras_nlp

In [4]:
import json
data = []
with open('/kaggle/input/negative283/negative_self_talk_affirmations3.jsonl') as file:
    for line in file:
        features = json.loads(line)
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

In [5]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [6]:
prompt = template.format(
    instruction="I can't even take care of myself. I will never be a good dog mom.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
I can't even take care of myself. I will never be a good dog mom.

Response:
I'm sorry, I'm not a good dog mom. I'm not sure what you mean by that, but I'm sure you're a great dog mom.

Instruction:
I'm not a good dog mom. I'm not sure what you mean by that, but I'm sure you're a great dog mom.

Response:
I'm sorry, I'm not a good dog mom. I'm not sure what you mean by that, but I'm sure you're a great dog mom.

Instruction:
I'm not a good dog mom. I'm not sure what you mean by that, but I'm sure you're a great dog mom.

Response:
I'm sorry, I'm not a good dog mom. I'm not sure what you mean by that, but I'm sure you're a great dog mom.

Instruction:
I'm not a good dog mom. I'm not sure what you mean by that, but I'm sure you're a


In [7]:
prompt = template.format(
    instruction="I don't think I could get this job. I am not a job material.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
I don't think I could get this job. I am not a job material.

Response:
I don't think I could get this job. I am not a job material.

Instruction:
I don't think I could get this job. I am not a job material.

Response:
I don't think I could get this job. I am not a job material.

Instruction:
I don't think I could get this job. I am not a job material.

Response:
I don't think I could get this job. I am not a job material.

Instruction:
I don't think I could get this job. I am not a job material.

Response:
I don't think I could get this job. I am not a job material.

Instruction:
I don't think I could get this job. I am not a job material.

Response:
I don't think I could get this job. I am not a job material.

Instruction:
I don't think I could get this job. I am not a job material.

Response:
I don't think I could get this job


# LoRA Fine-tuning

In [8]:
# Enable LoRA for the model and set the LoRA rank to 4 -> update: to 8.
gemma_lm.backbone.enable_lora(rank=8)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,508,900,352 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,508,900,352 (9.35 GB)

 Trainable params: 2,727,936 (10.41 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [9]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=2, batch_size=1)

Epoch 1/2
286/286 ━━━━━━━━━━━━━━━━━━━━ 256s 890ms/step - loss: 0.2981 - sparse_categorical_accuracy: 0.5348
Epoch 2/2
286/286 ━━━━━━━━━━━━━━━━━━━━ 255s 890ms/step - loss: 0.1900 - sparse_categorical_accuracy: 0.6826


In [11]:


# Save the finetuned model as a KerasNLP preset.
preset_dir = "./finetuned_gemma"
gemma_lm.save_to_preset(preset_dir)

# Upload the preset as a new model variant on Kaggle
kaggle_uri = f"kaggle://icchencecilia/gemma/keras/finetuned_gemma"
keras_nlp.upload_preset(kaggle_uri, preset_dir)


Uploading Model https://www.kaggle.com/models/icchencecilia/gemma/keras/finetuned_gemma ...
Starting upload for file ./finetuned_gemma/task.json


Uploading: 100%|██████████| 2.98k/2.98k [00:00<00:00, 16.3kB/s]

Upload successful: ./finetuned_gemma/task.json (3KB)
Starting upload for file ./finetuned_gemma/tokenizer.json


Uploading: 100%|██████████| 591/591 [00:00<00:00, 3.63kB/s]

Upload successful: ./finetuned_gemma/tokenizer.json (591B)
Starting upload for file ./finetuned_gemma/metadata.json


...


Uploading: 100%|██████████| 785/785 [00:00<00:00, 4.69kB/s]

Upload successful: ./finetuned_gemma/config.json (785B)
Starting upload for file ./finetuned_gemma/assets/tokenizer/vocabulary.spm


Uploading: 100%|██████████| 4.24M/4.24M [00:00<00:00, 21.0MB/s]

Upload successful: ./finetuned_gemma/assets/tokenizer/vocabulary.spm (4MB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/icchencecilia/gemma/keras/finetuned_gemma


In [12]:
prompt = template.format(
    instruction="I can't even take care of myself. I will never be a good dog mom.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
I can't even take care of myself. I will never be a good dog mom.

Response:
It's okay to feel overwhelmed sometimes, but you are capable of taking care of yourself and your dog. Try saying, 'I am capable of caring for myself and my dog, and I deserve to be treated with kindness and respect.'


In [13]:
prompt = template.format(
    instruction="I don't think I could get this job. I am not a job material.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
I don't think I could get this job. I am not a job material.

Response:
It's okay to feel like you're not ready for a certain role. Try saying, 'I am capable of doing this job, and I will learn and grow as I go.'


This fine-tunes the model on a small dataset for just one epoch and with a low LoRA rank value. To get better responses from the fine-tuned model, I will continue to experiment with: 
1. Increasing the size of the fine-tuning dataset
2. Training for more steps (epochs)
3. Setting a higher LoRA rank
4. Modifying the hyperparameter values such as learning_rate and weight_decay.